# Launch Daily Loop

In [1]:
import os, sys, random, json, datetime
import pandas as PD

In [2]:
BACKEND_PATH = os.path.dirname(os.path.dirname(os.getcwd()))
APPPROJ_PATH = os.path.join(BACKEND_PATH, 'app_proj')
sys.path.append(APPPROJ_PATH)
import notebooks as NT
print([x for x in dir(NT) if x[0]!='_' and len(x)>8])

['DataframeToDicts', 'GetRandom', 'NOTEBOOK_ENV']


In [3]:
import django
os.environ['DEPLOYMENT_ENV'] = NT.NOTEBOOK_ENV
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'True'
MODULE_PATH = os.path.dirname(os.getcwd())
os.chdir(os.path.dirname(MODULE_PATH))
django.setup()

env: DEV


In [4]:
from django.utils import timezone

In [5]:
import database as DB
print([x for x in dir(DB) if x[0]!='_' and len(x)>8])

['BaseManager', 'DeleteTable', 'GetNativeTableCounts', 'GetTableCounts', 'GetTableDictionary', 'InsertBulk', 'InsertSingle']


In [6]:
import django.db.models as JM
from django.contrib.auth import get_user_model

In [7]:
import emporium.models as EM

import engine.models as GM
import engine.logic.content as CT
import engine.logic.launcher as LC
import engine.logic.simulation as SM

In [8]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

### Guild Properties

In [9]:
userMd = get_user_model().objects.filter(user_name='Admin')[0]
userMd

<User: admin@thieves-guild.com>

In [10]:
guild = GM.Guild.objects.GetOrNone(UserFK=userMd, Name='123')
PD.DataFrame([guild.__dict__]).drop(['id', '_state'], axis=1, errors='ignore')

,UserFK_id,Name,KeepLevel,TotalPower,VaultGold,VaultGems,Selected,LastPlayed,CreateDate
0,1,123,1,0,0,0,True,2023-08-11,2023-08-11


In [11]:
thieves = GM.ThiefInGuild.objects.filter(GuildFK=guild).values()
PD.DataFrame(thieves).drop(['id',
        'Name', 'BasePower', 'Experience', 
        'Position', 
        'BaseAgi', 'BaseCun', 'BaseMig', 'BaseEnd', 
        'TrainedAgi', 'TrainedCun', 'TrainedMig', 'TrainedEnd', 
        'Sabotage', 'Perceive', 'Traverse',
        ], axis=1, errors='ignore')

,GuildFK_id,Class,Stars,Level,Power,Agility,Cunning,Might,Endurance,Health,Attack,Damage,Defense,Wounds,Cooldown
0,15,Burglar,1,1,70,5,0,0,0,70,5,11,13,0,None
1,15,Scoundrel,1,1,70,0,5,0,0,70,2,15,12,0,None
2,15,Ruffian,1,1,70,0,0,5,0,70,3,10,16,0,None


In [12]:
towers = GM.GuildTower.objects.all().values()
PD.DataFrame(towers).drop(['id'], axis=1, errors='ignore')[4:9]

""


### Robot Loop

In [19]:
# prepare daily content

SM.ResetAllThieves(guild)
CT.SetGuildTower(guild)

# assign equipment

SM.RefreshEquipment(guild)

# run league trial after keep 3

# run campaign, one trial only, on success try another


# run gothic tower

stopLoop = 0
while True and stopLoop < 10:

    print('-------------')
    
    # get thief assignments
    
    nextStage = SM.GetNextTower(guild)
    PD.DataFrame([nextStage.__dict__]).drop(['id', '_state', 'GuildFK_id'], axis=1, errors='ignore')

    thieves = list(GM.ThiefInGuild.objects.filter(GuildFK=guild).order_by('Power'))
    readyLs = [x for x in thieves if x.Cooldown is None]

    stagePlays = SM.GetPlayAssignments(readyLs, nextStage)
    if not stagePlays: break

    # launch current stage
    
    res1, res2, res3 = LC.LaunchStage(stagePlays, nextStage)
    
    PD.DataFrame(res1)
    if res2: PD.DataFrame(res2)
    if res3: PD.DataFrame(res3)

    # record results
    
    rollParams = [x['rollParams'] for x in res1]
    stagePlays[0]['obstacleResults'] = rollParams
    stagePlays[0]['wounds'] = stagePlays[0]['thief'].Wounds
    stagePlays[0]['complete'] = (res1[-1]['currPos'], res1[-1]['nextPos'])
    
    
    stagePlays[0]['thief'].Cooldown = timezone.now() + datetime.timedelta(hours=1)
    stagePlays[0]['thief'].save()
    nextStage.CompleteR1 = True

    if len(stagePlays) >= 2:
        stagePlays[1]['thief'].Cooldown = timezone.now() + datetime.timedelta(hours=1)
        stagePlays[1]['thief'].save()
        nextStage.CompleteR2 = True

    if len(stagePlays) == 3:
        stagePlays[2]['thief'].Cooldown = timezone.now() + datetime.timedelta(hours=1)
        stagePlays[2]['thief'].save()
        nextStage.CompleteR3 = True

    nextStage.save()

    PD.DataFrame(stagePlays)

    stopLoop += 1


-------------


,StageNo,StageDate,ObstaclesR1,ObstaclesR2,ObstaclesR3,TypeR1,TypeR2,TypeR3,CompleteR1,CompleteR2,CompleteR3
0,1,2023-08-12,"[{""Level"": 1, ""Name"": ""Barrel"", ""Trait"": ""Mig""...",None,None,balanced,None,None,False,None,None


,obstacle,currPos,nextPos,rollParams,reward,wounds
0,Barrel,0,1,"{'roll': 20, 'traitBonus': 0, 'skillBonus': 0,...",xp 8,NaN
1,Crossbow Trap,1,2,"{'roll': 12, 'traitBonus': 0, 'skillBonus': 5,...",xp 8,NaN
2,Spike Trap,2,3,"{'roll': 5, 'traitBonus': 5, 'skillBonus': 0, ...",None,6.0
3,Warden,3,4,"[{'attacker': 'thief', 'roll': 9, 'attack': 5,...",xp 13,6.0
4,Chest,4,5,"{'roll': 10, 'traitBonus': 5, 'skillBonus': 0,...",None,NaN
5,Arcane Seal,5,6,"{'roll': 8, 'traitBonus': 0, 'skillBonus': 0, ...",None,17.0


,thief,room,obstacleResults,roomResults,wounds,complete
0,Burglar 70 1,1,"[{'roll': 20, 'traitBonus': 0, 'skillBonus': 0...",{},29,"(5, 6)"


-------------


...

,StageNo,StageDate,ObstaclesR1,ObstaclesR2,ObstaclesR3,TypeR1,TypeR2,TypeR3,CompleteR1,CompleteR2,CompleteR3
0,3,2023-08-12,"[{""Level"": 1, ""Name"": ""Sorcerer"", ""Trait"": ""An...",None,None,balanced,None,None,False,None,None


,obstacle,currPos,nextPos,rollParams,reward,wounds
0,Sorcerer,0,1,"[{'attacker': 'thief', 'roll': 15, 'attack': 3...",xp 13,0.0
1,Crossbow Trap,1,2,"{'roll': 7, 'traitBonus': 0, 'skillBonus': 0, ...",None,15.0
2,Idol,2,3,"{'roll': 9, 'traitBonus': 5, 'skillBonus': 0, ...",None,NaN
3,Door,3,4,"{'roll': 9, 'traitBonus': 0, 'skillBonus': 0, ...",None,16.0
4,Secret Passage,4,6,"{'roll': 19, 'traitBonus': 0, 'skillBonus': 0,...",xp 8,NaN
5,Crossbow Trap,6,7,"{'roll': 13, 'traitBonus': 0, 'skillBonus': 0,...",xp 8,NaN
6,Chest,7,8,"{'roll': 11, 'traitBonus': 0, 'skillBonus': 0,...",None,NaN


,thief,room,obstacleResults,roomResults,wounds,complete
0,Ruffian 70 1,1,"[[{'attacker': 'thief', 'roll': 15, 'attack': ...",{},31,"(7, 8)"


-------------


,StageNo,StageDate,ObstaclesR1,ObstaclesR2,ObstaclesR3,TypeR1,TypeR2,TypeR3,CompleteR1,CompleteR2,CompleteR3
0,4,2023-08-12,"[{""Level"": 1, ""Name"": ""Sorcerer"", ""Trait"": ""An...",None,None,biased agi,None,None,False,None,None


In [14]:
thieves = GM.ThiefInGuild.objects.filter(GuildFK=guild).values()
PD.DataFrame(thieves).drop(['id',
        'Name', 'BasePower', 'Experience', 
        'Position', 
        'BaseAgi', 'BaseCun', 'BaseMig', 'BaseEnd', 
        'TrainedAgi', 'TrainedCun', 'TrainedMig', 'TrainedEnd', 
        'Sabotage', 'Perceive', 'Traverse',
        ], axis=1, errors='ignore')

,GuildFK_id,Class,Stars,Level,Power,Agility,Cunning,Might,Endurance,Health,Attack,Damage,Defense,Wounds,Cooldown
0,15,Burglar,1,1,70,5,0,0,0,70,5,11,13,28,2023-08-12 01:17:55.596073+00:00
1,15,Scoundrel,1,1,70,0,5,0,0,70,2,15,12,32,2023-08-12 01:17:55.649437+00:00
2,15,Ruffian,1,1,70,0,0,5,0,70,3,10,16,9,2023-08-12 01:17:55.542684+00:00


In [15]:
stagePlayLs = [{
    'thief': None,
    'room': 0,              
    'obstacleResults': [],    # results of each obstacle, display as thief goes through
    'roomResults': {},       # compile from obstacles, display at end stage
                              # end stage results?
}]